# Simulação de aplicação

O paradigma SSVEP é bastante utilizado em aplicações reais por ser fácil de evocar os sinais e posteriormente classificá-los.

Para processar uma aplicação SSVEP em tempo real é necessário calcular a máxima energia dos _targets_ em janelas ou _buffers_ suficientemente grandes. O tamanho da janela de dados é inversamente proporcional a taxa de amostragem do equipamente de EEG utilizado. Ou seja, quanto maior a taxa de amostragem, menor o tempo decessário para calcular a **energia deslizante do sinal**.

**Tarefa**: simule uma aplicação SSVEP criando um buffer do sinal no domínio do tempo em uma única dimensão. Se inicialmente a dimensão dos dados era `(100, 8192)`, agora será `(819200)`. Crie um buffer de 3 ou 4 segundos, informando a cada segundo qual está sendo a frequência evocada.

In [12]:
import numpy as np
import mne

fs = 8192
duration = 10
buffer_size = 3
step_size = 1
n_samples = fs * duration
n_buffer_samples = fs * buffer_size
n_step_samples = fs * step_size

time = np.arange(n_samples) / fs
signal = np.sin(2 * np.pi * 10 * time) + 0.5 * np.sin(2 * np.pi * 15 * time)

np.random.seed(42)
signal += 0.1 * np.random.randn(n_samples)
signal = signal.flatten()

def process_buffer(buffer, fs, fmin=3, fmax=30):
    psd, freqs = mne.time_frequency.psd_array_multitaper(buffer, sfreq=fs, fmin=fmin, fmax=fmax, verbose=False)
    psd_values = psd
    max_idx = np.argmax(psd_values)
    max_freq = freqs[max_idx]
    return max_freq

results = []

for start_idx in range(0, len(signal) - n_buffer_samples, n_step_samples):
    end_idx = start_idx + n_buffer_samples
    buffer = signal[start_idx:end_idx]
    max_freq = process_buffer(buffer, fs)
    results.append((start_idx / fs, end_idx / fs, max_freq))

print(f"{'Start(s)':<15}{'End(s)':<15}{'FrequenciaEvocada(Hz)':<20}")
for start_time, end_time, evoked_freq in results:
    print(f"{start_time:<15.2f}{end_time:<15.2f}{evoked_freq:<20.2f}")


Start(s)       End(s)         FrequenciaEvocada(Hz)
0.00           3.00           10.00               
1.00           4.00           9.67                
2.00           5.00           10.00               
3.00           6.00           9.67                
4.00           7.00           9.67                
5.00           8.00           9.67                
6.00           9.00           9.67                
